# 1. OpenWeathermap을 활용해 데이터 추출과정 만들기

- OpenWeather의 날씨 갱신 주기는 무료는 2~4시간 정도이다. 그러면 여유 있게 4시간을 기준으로 데이터를 추출해오는 방법을 생각
- 여기서 날씨 데이터를 가져올 기준을 정해야함
- 특정 도시를 무작위로 가져올 수 있도록 하고 싶긴함

## 1-1. API로 데이터 가져와보기 (부산)

In [13]:
from config import api_key
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
city = "Busan"
base_url = "http://api.openweathermap.org/data/2.5/weather"

params = {
    "q": city,
    "appid": api_key,
}

response = requests.get(base_url, params=params)
data = response.json()

In [22]:
# Unix 타임 스탬프를 년, 월, 일, 시, 분, 초로 변환
timestamp = data['dt']
dt = datetime.utcfromtimestamp(timestamp)
formatted_datetime = dt.strftime('%Y-%m-%d %H:%M:%S')

# 'timezone' 값을 UTC 기준으로 9시간 뒤로 조정
timezone_offset_seconds = 9 * 3600  # 9시간을 초 단위로 변환
adjusted_datetime = dt + timedelta(seconds=timezone_offset_seconds)
formatted_adjusted_datetime = adjusted_datetime.strftime('%Y-%m-%d %H:%M:%S')

# 변환된 날짜 및 시간을 데이터에 추가
data['formatted_dt'] = formatted_datetime
data['formatted_adjusted_dt'] = formatted_adjusted_datetime

# 필요한 필드 추출
selected_data = {
    'City ID': data['id'],
    'City Name': data['name'],
    'Lon': data['coord']['lon'],
    'Lat': data['coord']['lat'],
    'Weather ID': data['weather'][0]['id'],
    'Weather Main': data['weather'][0]['main'],
    'Temperature (K)' : data['main']['temp'],
    'Humidity (%)' : data['main']['humidity'],
    'Visibility (m)': data['visibility'],
    'Wind Speed (m/s)': data['wind']['speed'],
    'Clouds (%)': data['clouds']['all'],
    'Date (Unix)': data['formatted_dt'],
    'Timezone (s)': data['formatted_adjusted_dt'],
}

# 데이터프레임 생성
df = pd.DataFrame([selected_data])

In [23]:
# 도시ID, 도시이름, 경도, 위도, 날씨ID, 날씨, 온도, 습도, 가시거리, 풍속, 구름 양, 데이터 시간, 지역 시간대
df

,City ID,City Name,Lon,Lat,Weather ID,Weather Main,Temperature (K),Humidity (%),Visibility (m),Wind Speed (m/s),Clouds (%),Date (Unix),Timezone (s)
0,1838524,Busan,129.0403,35.1028,801,Clouds,297.14,69,10000,3.6,20,2023-09-07 11:49:03,2023-09-07 20:49:03


In [26]:
df.columns

Index(['City ID', 'City Name', 'Lon', 'Lat', 'Weather ID', 'Weather Main',
       'Temperature (K)', 'Humidity (%)', 'Visibility (m)', 'Wind Speed (m/s)',
       'Clouds (%)', 'Date (Unix)', 'Timezone (s)'],
      dtype='object')

In [27]:
df.dtypes

City ID               int64
City Name            object
Lon                 float64
Lat                 float64
Weather ID            int64
Weather Main         object
Temperature (K)     float64
Humidity (%)          int64
Visibility (m)        int64
Wind Speed (m/s)    float64
Clouds (%)            int64
Date (Unix)          object
Timezone (s)         object
dtype: object

## 1-2. 부산의 최근 한달간 날씨 정보

In [29]:
# 도시 이름
city = "Busan"

# OpenWeatherMap API 엔드포인트
base_url = "http://api.openweathermap.org/data/2.5/onecall"

# 현재 시간 구하기
current_time = datetime.now()

# 한 달 전의 날짜 계산
one_month_ago = current_time - timedelta(days=30)
unix_timestamp = int(one_month_ago.timestamp())

# API 요청 파라미터 설정
params = {
    "lat": 35.1028,  # 부산의 위도 정보
    "lon": 129.0403,  # 부산의 경도 정보
    "exclude": "current,minutely,hourly",  # 현재, 분 단위, 시간 단위 날씨 정보 제외
    "appid": api_key,
}

# API 요청 보내기
response = requests.get(base_url, params=params)
data = response.json()

# 필요한 데이터 추출 (예: 일일 날씨 정보)
daily_weather_data = data.get("daily", [])

# 데이터프레임으로 변환
weather_df = pd.DataFrame(daily_weather_data)

weather_df
# Unix 타임스탬프를 년-월-일 형식의 날짜로 변환
#weather_df["Date"] = weather_df["dt"].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))

# 필요한 열 선택 (예: 날짜, 최저 기온, 최고 기온, 습도, 날씨 상태 등)
#selected_columns = ["Date", "temp_min", "temp_max", "humidity", "weather"]
#weather_df = weather_df[selected_columns]

# 데이터프레임 출력
#print(weather_df)

,dt,sunrise,sunset,moonrise,moonset,moon_phase,temp,feels_like,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,weather,clouds,pop,uvi,rain
0,1694055600,1694034023,1694079857,1694096340,1694061960,0.75,"{'day': 299.94, 'min': 295.25, 'max': 300.27, ...","{'day': 301.04, 'night': 297.38, 'eve': 298.53...",1014,61,291.60,9.57,48,12.56,"[{'id': 800, 'main': 'Clear', 'description': '...",8,0.00,7.82,NaN
1,1694142000,1694120466,1694166172,0,1694151900,0.79,"{'day': 299.37, 'min': 295.37, 'max': 299.62, ...","{'day': 299.37, 'night': 295.61, 'eve': 298.08...",1013,57,289.80,7.72,55,11.08,"[{'id': 800, 'main': 'Clear', 'description': '...",6,0.00,7.09,NaN
2,1694228400,1694206910,1694252487,1694185800,1694241420,0.82,"{'day': 299.17, 'min': 293.89, 'max': 299.63, ...","{'day': 299.17, 'night': 295.98, 'eve': 298.14...",1012,59,290.02,5.96,55,7.18,"[{'id': 800, 'main': 'Clear', 'description': '...",2,0.00,8.22,NaN
3,1694314800,1694293353,1694338801,1694275560,1694330460,0.85,"{'day': 299.37, 'min': 294.58, 'max': 300.08, ...","{'day': 299.37, 'night': 296.95, 'eve': 298.32...",1010,67,292.53,7.58,52,9.37,"[{'id': 500, 'main': 'Rain', 'description': 'l...",75,0.91,5.47,1.91
4,1694401200,1694379796,1694425115,1694365440,1694419140,0.88,"{'day': 297.31, 'min': 295.53, 'max': 298.12, ...","{'day': 297.9, 'night': 297.37, 'eve': 298.26,...",1008,81,293.69,5.99,55,7.01,"[{'id': 500, 'main': 'Rain', 'description': 'l...",100,0.83,7.76,3.07
5,1694487600,1694466239,1694511428,1694455440,1694507460,0.91,"{'day': 299.57, 'min': 295.93, 'max': 299.57, ...","{'day': 299.57, 'night': 298.65, 'eve': 299.85...",1008,76,294.70,3.98,39,4.74,"[{'id': 500, 'main': 'Rain', 'description': 'l...",71,0.66,8.00,1.25
6,1694574000,1694552683,1694597742,1694545380,1694595540,0.94,"{'day': 300.76, 'min': 296.81, 'max': 300.76, ...","{'day': 302.97, 'night': 298.3, 'eve': 299.33,...",1009,69,294.18,5.65,45,8.02,"[{'id': 500, 'main': 'Rain', 'description': 'l...",97,1.00,8.00,4.02
7,1694660400,1694639126,1694684055,1694635260,1694683440,0.97,"{'day': 295.23, 'min': 294.92, 'max': 296.71, ...","{'day': 295.77, 'night': 296.72, 'eve': 296.76...",1014,87,292.76,9.70,43,14.54,"[{'id': 501, 'main': 'Rain', 'description': 'm...",100,1.00,8.00,11.85


## 1-3. OpenWeather에서 제공하는 도시 이름 전부 가져오기

In [40]:
import json

json_file_path = "./files/city.list.json"

with open(json_file_path, encoding='utf-8') as json_file:
    city_data = json.load(json_file)


In [41]:
len(city_data)

209579

In [42]:
city_data[0]

{'id': 833,
 'name': 'Ḩeşār-e Sefīd',
 'state': '',
 'country': 'IR',
 'coord': {'lon': 47.159401, 'lat': 34.330502}}

## 1-4. Openweather에서 제공하는 도시 목록별 데이터 프레임화 파이프라인

In [47]:
today_data = []

for i in range(100):
    city = city_data[i]['name']
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": city,
        "appid": api_key,
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Unix 타임 스탬프를 년, 월, 일, 시, 분, 초로 변환
    timestamp = data['dt']
    dt = datetime.utcfromtimestamp(timestamp)
    formatted_datetime = dt.strftime('%Y-%m-%d %H:%M:%S')

    # 'timezone' 값을 UTC 기준으로 9시간 뒤로 조정
    timezone_offset_seconds = 9 * 3600  # 9시간을 초 단위로 변환
    adjusted_datetime = dt + timedelta(seconds=timezone_offset_seconds)
    formatted_adjusted_datetime = adjusted_datetime.strftime('%Y-%m-%d %H:%M:%S')

    # 변환된 날짜 및 시간을 데이터에 추가
    data['formatted_dt'] = formatted_datetime
    data['formatted_adjusted_dt'] = formatted_adjusted_datetime

    # 필요한 필드 추출
    selected_data = {
        'City ID': data['id'],
        'City Name': data['name'],
        'Lon': data['coord']['lon'],
        'Lat': data['coord']['lat'],
        'Weather ID': data['weather'][0]['id'],
        'Weather Main': data['weather'][0]['main'],
        'Temperature (K)' : data['main']['temp'],
        'Humidity (%)' : data['main']['humidity'],
        'Visibility (m)': data['visibility'],
        'Wind Speed (m/s)': data['wind']['speed'],
        'Clouds (%)': data['clouds']['all'],
        'Date (Unix)': data['formatted_dt'],
        'Timezone (s)': data['formatted_adjusted_dt'],
    }
    
    today_data.append(selected_data)

# 데이터프레임 생성
df = pd.DataFrame(today_data)

In [48]:
df

,City ID,City Name,Lon,Lat,Weather ID,Weather Main,Temperature (K),Humidity (%),Visibility (m),Wind Speed (m/s),Clouds (%),Date (Unix),Timezone (s)
0,833,Ḩeşār-e Sefīd,47.1594,34.3305,800,Clear,310.17,1,10000,5.14,0,2023-09-08 12:36:09,2023-09-08 21:36:09
1,2960,‘Ayn Ḩalāqīm,36.3219,34.9401,800,Clear,305.76,21,10000,6.21,0,2023-09-08 12:36:09,2023-09-08 21:36:09
2,3245,Taglag,44.9833,38.4500,800,Clear,305.31,19,10000,3.44,0,2023-09-08 12:36:10,2023-09-08 21:36:10
3,3530,Qabāghlū,46.1685,36.1733,800,Clear,303.58,7,10000,7.56,0,2023-09-08 12:36:11,2023-09-08 21:36:11
4,5174,‘Arīqah,36.4834,32.8898,800,Clear,309.08,11,10000,7.05,0,2023-09-08 12:36:11,2023-09-08 21:36:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,58802,Gedo,42.0000,3.0000,803,Clouds,307.41,30,10000,8.15,60,2023-09-08 12:37:02,2023-09-08 21:37:02
96,58933,Garoowe,48.4845,8.4054,804,Clouds,307.65,30,10000,9.86,95,2023-09-08 12:37:02,2023-09-08 21:37:02
97,58994,Garbahaarrey,42.2209,3.3289,803,Clouds,308.99,28,10000,7.99,61,2023-09-08 12:37:03,2023-09-08 21:37:03
98,59362,Galguduud,47.0000,5.0000,801,Clouds,309.86,25,10000,8.71,23,2023-09-08 12:37:03,2023-09-08 21:37:03


In [50]:
# 데이터프레임을 CSV 파일로 저장
df.to_csv('./files/20230908_Weather.csv', index=False)  # index를 포함하지 않으려면 index=False를 사용하세요